## R12725049 Assignment2

### 1(a.) 試簡述何謂維度的詛咒？試列舉一案例說明。

- 隨著函數（或模型）所需要估計未知參數的增加，樣本所需要的數量會呈現指數型的成長，樣本數不足的情況容易發生不易收斂或多重解的可能性
- 如影像處理即可能面臨維度詛咒。一個800*600的彩色照片代表有1440000個變數。

### 1(b.) 避免維度詛咒的方法有哪些？

- 降低維度
- 在給定誤差的情況下滿足最少需要的樣本數
- 僅留下需要的Feature
- Clustering

### 1(c.)，試著重新繪製圖 3.12，呈現維度與預測

### 2(a.) 試說明損失函數與模型評估指標有何不同？

#### 模型訓練時使用損失函數，模型訓練後使用評估指標
- 損失函數
    - 訓練與配適模型時最小化的目標函數，作為調整模型複雜度與權衡偏誤與變異
    - 其函數特性常需要滿足一定的要求，例如具備「微分特性」、「連續性」等性質。通常只能比較相對性，無法解釋絕對的誤差大小
- 評估指標
    - 用以衡量訓練後的模型表現，評估最終結果，需具備「公平性」與「比較性」

### 2(b.)試使用網際網路(internet)學習，損失函數的設計有哪些？試列舉兩種，並說明其各自的優缺點或可建議的使用時機。

MSE與Log Loss是兩種常見的Loss Funtion
- MSE：常用於Lenear Regression的問題上
    - 優點：相對容易理解和計算，因為它具有可微分的圖形。
    - 缺點：對異常值不敏感，若誤差小於一，可能低估模型的不良程度。
- Log Loss：常用於Logistic Regression的問題上
    - 優點：非常適用於大量觀測數據，當觀測數量達到數千以上時，表現非常出色。
    - 缺點：不像MSE那樣容易解釋和計算，尤其在多分類問題中，且對異常值敏感。

### 3(a) 根據此開放數據，您會用什麼方法來確認資料品質的好壞？試操作一次並說明其細節。

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [2]:
# 使用Pandas的read_excel函數讀取檔案
# 因沒有表頭所以先加入虛擬表頭
file_path = r"./MDS_Assignment2_kidney.xlsx"
df_kidney = pd.read_excel(file_path,sheet_name=0,header=None)
df_kidney

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,48,80,1.02,1,0,?,normal,notpresent,notpresent,121,36,1.2,?,?,15.4,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7,50,1.02,4,0,?,normal,notpresent,notpresent,?,18,0.8,?,?,11.3,38,6000,?,no,no,no,good,no,no,ckd
2,62,80,1.01,2,3,normal,normal,notpresent,notpresent,423,53,1.8,?,?,9.6,31,7500,?,no,yes,no,poor,no,yes,ckd
3,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,56,3.8,111,2.5,11.2,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51,80,1.01,2,0,normal,normal,notpresent,notpresent,106,26,1.4,?,?,11.6,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55,80,1.02,0,0,normal,normal,notpresent,notpresent,140,49,0.5,150,4.9,15.7,47,6700,4.9,no,no,no,good,no,no,notckd
396,42,70,1.025,0,0,normal,normal,notpresent,notpresent,75,31,1.2,141,3.5,16.5,54,7800,6.2,no,no,no,good,no,no,notckd
397,12,80,1.02,0,0,normal,normal,notpresent,notpresent,100,26,0.6,137,4.4,15.8,49,6600,5.4,no,no,no,good,no,no,notckd
398,17,60,1.025,0,0,normal,normal,notpresent,notpresent,114,50,1,135,4.9,14.2,51,7200,5.9,no,no,no,good,no,no,notckd


In [3]:
# 使用Pandas的read_excel函數讀取第二個Sheet做為表頭
df = pd.read_excel(file_path,sheet_name=1,usecols="D",header=None)
new_df = df[31:56]
new_df

,3
31,age
32,bp
33,sg
34,al
35,su
36,rbc
37,pc
38,pcc
39,ba
40,bgr


In [4]:
df_kidney.columns = new_df.T.values[0]

In [5]:
# 查看資料及概況
df_kidney.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   age     400 non-null    object
 1   bp      400 non-null    object
 2   sg      400 non-null    object
 3   al      400 non-null    object
 4   su      400 non-null    object
 5   rbc     400 non-null    object
 6   pc      400 non-null    object
 7   pcc     400 non-null    object
 8   ba      400 non-null    object
 9   bgr     400 non-null    object
 10  bu      400 non-null    object
 11  sc      400 non-null    object
 12  sod     400 non-null    object
 13  pot     400 non-null    object
 14  hemo    400 non-null    object
 15  pcv     398 non-null    object
 16  wc      395 non-null    object
 17  rc      394 non-null    object
 18  htn     394 non-null    object
 19  dm      389 non-null    object
 20  cad     387 non-null    object
 21  appet   387 non-null    object
 22  pe      387 non-null    ob

In [6]:
df_kidney.describe()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,class
count,400,400,400.00,400,400,400,400,400,400,400,400,400.0,400,400,400,398,395,394,394,389,387,387,387,387,387
unique,77,11,6.00,7,7,3,3,3,3,147,119,85.0,35,41,116,43,89,45,3,4,3,3,3,3,2
top,60,80,1.02,0,0,normal,normal,notpresent,notpresent,?,?,1.2,?,?,?,?,?,?,no,no,no,good,no,no,ckd
freq,19,116,106.00,199,290,201,259,354,374,44,19,40.0,87,88,52,70,104,128,247,256,351,304,312,326,237


**上述檢查數據準確性、一致性，可看出某些數值有明顯的outliers**

In [7]:
# 查看資料內有多少空值與問號值
df_kidney.replace('?', np.nan, inplace=True)
df_kidney.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       72
wc       109
rc       134
htn        8
dm        13
cad       15
appet     14
pe        14
ane       14
class     13
dtype: int64

**上述檢查數據完整性，可看出幾乎每列都有空值，完整性不佳**

In [8]:
# cat to num:
columnsname=df_kidney.columns
# len(columnsname)
for i in range (0,25):
    print(columnsname[i],df_kidney[columnsname[i]].unique())

age [48.  7. 62. 51. 60. 68. 24. 52. 53. 50. 63. 40. 47. 61. 21. 42. 75. 69.
 nan 73. 70. 65. 76. 72. 82. 46. 45. 35. 54. 11. 59. 67. 15. 55. 44. 26.
 64. 56.  5. 74. 38. 58. 71. 34. 17. 12. 43. 41. 57.  8. 39. 66. 81. 14.
 27. 83. 30.  4.  3.  6. 32. 80. 49. 90. 78. 19.  2. 33. 36. 37. 23. 25.
 20. 29. 28. 22. 79.]
bp [ 80.  50.  70.  90.  nan 100.  60. 110. 140. 180. 120.]
sg [1.02  1.01  1.005 1.015   nan 1.025]
al [ 1.  4.  2.  3.  0. nan  5.]
su [ 0.  3.  4.  1. nan  2.  5.]
rbc [nan 'normal' 'abnormal']
pc ['normal' 'abnormal' nan]
pcc ['notpresent' 'present' nan]
ba ['notpresent' 'present' nan]
bgr [121.  nan 423. 117. 106.  74. 100. 410. 138.  70. 490. 380. 208.  98.
 157.  76.  99. 114. 263. 173.  95. 108. 156. 264. 123.  93. 107. 159.
 140. 171. 270.  92. 137. 204.  79. 207. 124. 144.  91. 162. 246. 253.
 141. 182.  86. 150. 146. 425. 112. 250. 360. 163. 129. 133. 102. 158.
 165. 132. 104. 127. 415. 169. 251. 109. 280. 210. 219. 295.  94. 172.
 101. 298. 153.  88. 226. 143. 1

In [9]:
df_kidney['dm'] = df_kidney['dm'].replace(to_replace={' yes':'yes'})

### 3(b)試建議三個可能衡量數據品質的量化指標(i.e. KPIs)

1. Error Rate：數據中錯誤值的比例，用以衡量Data Accuracy
2. Missing Value Ratio：數值中遺漏值的百分比，用以衡量Data Completeness
3. Duplicate Data Ratio：數值中重複資料的比例，用以衡量Data Consistency

### 3(c)如何處理遺漏值(missing values)？又或某些欄位不打算遺漏值處理的理由為何？

In [10]:
# 將欄位分為numerical以及categorical以便後續處理
columns = df_kidney.columns
columns_numerical = ["age", "bp", "sg", "al", "su", "bgr", "bu", "sc", "sod", "pot", "hemo", "pcv", "wc", "rc"]
columns_categorical = ["rbc", "pc", "pcc", "ba", "htn", "dm", "cad", "appet", "pe", "ane", "class"]

print("Atribut Numerical: {}".format(columns_numerical))
print("Atribut Categorical: {}".format(columns_categorical))
print("Panjang Atribut Numerical dan Categorical: {}".format(len(columns_numerical)+len(columns_categorical)))

Atribut Numerical: ['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc']
Atribut Categorical: ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'class']
Panjang Atribut Numerical dan Categorical: 25


In [11]:
# change df classficication to 0 and 1
df_kidney['class'] = df_kidney['class'].replace(to_replace={'ckd':1, 'notckd':0})

In [12]:
print("Dataset Original")
display(df_kidney)
nan_values = df_kidney[columns_categorical].isnull().sum()

simpleImputer = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
df_kidney[columns_categorical] = simpleImputer.fit_transform(df_kidney[columns_categorical])

print("利用SimpleImputer填補類別欄位空值")
display(df_kidney)

encoder = LabelEncoder()
df_kidney[columns_categorical] = df_kidney[columns_categorical].apply(encoder.fit_transform)

print("利用LabelEncoder為字串的類別欄位編碼")
display(df_kidney)

imputer = KNNImputer(n_neighbors=5)
df_kidney[columns_numerical] = imputer.fit_transform(df_kidney[columns_numerical])

print("利用KNN填補數值欄位空值")
display(df_kidney)


Dataset Original


,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,36.0,1.2,NaN,NaN,15.4,44.0,7800.0,5.2,yes,yes,no,good,no,no,1.0
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,18.0,0.8,NaN,NaN,11.3,38.0,6000.0,NaN,no,no,no,good,no,no,1.0
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,53.0,1.8,NaN,NaN,9.6,31.0,7500.0,NaN,no,yes,no,poor,no,yes,1.0
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,56.0,3.8,111.0,2.5,11.2,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,1.0
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,26.0,1.4,NaN,NaN,11.6,35.0,7300.0,4.6,no,no,no,good,no,no,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,49.0,0.5,150.0,4.9,15.7,47.0,6700.0,4.9,no,no,no,good,no,no,0.0
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,31.0,1.2,141.0,3.5,16.5,54.0,7800.0,6.2,no,no,no,good,no,no,0.0
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,26.0,0.6,137.0,4.4,15.8,49.0,6600.0,5.4,no,no,no,good,no,no,0.0
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,50.0,1.0,135.0,4.9,14.2,51.0,7200.0,5.9,no,no,no,good,no,no,0.0


利用SimpleImputer填補類別欄位空值


,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,normal,normal,notpresent,notpresent,121.0,36.0,1.2,NaN,NaN,15.4,44.0,7800.0,5.2,yes,yes,no,good,no,no,1.0
1,7.0,50.0,1.020,4.0,0.0,normal,normal,notpresent,notpresent,NaN,18.0,0.8,NaN,NaN,11.3,38.0,6000.0,NaN,no,no,no,good,no,no,1.0
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,53.0,1.8,NaN,NaN,9.6,31.0,7500.0,NaN,no,yes,no,poor,no,yes,1.0
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,56.0,3.8,111.0,2.5,11.2,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,1.0
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,26.0,1.4,NaN,NaN,11.6,35.0,7300.0,4.6,no,no,no,good,no,no,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,49.0,0.5,150.0,4.9,15.7,47.0,6700.0,4.9,no,no,no,good,no,no,0.0
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,31.0,1.2,141.0,3.5,16.5,54.0,7800.0,6.2,no,no,no,good,no,no,0.0
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,26.0,0.6,137.0,4.4,15.8,49.0,6600.0,5.4,no,no,no,good,no,no,0.0
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,50.0,1.0,135.0,4.9,14.2,51.0,7200.0,5.9,no,no,no,good,no,no,0.0


利用LabelEncoder為字串的類別欄位編碼


,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,1,1,0,0,121.0,36.0,1.2,NaN,NaN,15.4,44.0,7800.0,5.2,1,1,0,0,0,0,1
1,7.0,50.0,1.020,4.0,0.0,1,1,0,0,NaN,18.0,0.8,NaN,NaN,11.3,38.0,6000.0,NaN,0,0,0,0,0,0,1
2,62.0,80.0,1.010,2.0,3.0,1,1,0,0,423.0,53.0,1.8,NaN,NaN,9.6,31.0,7500.0,NaN,0,1,0,1,0,1,1
3,48.0,70.0,1.005,4.0,0.0,1,0,1,0,117.0,56.0,3.8,111.0,2.5,11.2,32.0,6700.0,3.9,1,0,0,1,1,1,1
4,51.0,80.0,1.010,2.0,0.0,1,1,0,0,106.0,26.0,1.4,NaN,NaN,11.6,35.0,7300.0,4.6,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,1,1,0,0,140.0,49.0,0.5,150.0,4.9,15.7,47.0,6700.0,4.9,0,0,0,0,0,0,0
396,42.0,70.0,1.025,0.0,0.0,1,1,0,0,75.0,31.0,1.2,141.0,3.5,16.5,54.0,7800.0,6.2,0,0,0,0,0,0,0
397,12.0,80.0,1.020,0.0,0.0,1,1,0,0,100.0,26.0,0.6,137.0,4.4,15.8,49.0,6600.0,5.4,0,0,0,0,0,0,0
398,17.0,60.0,1.025,0.0,0.0,1,1,0,0,114.0,50.0,1.0,135.0,4.9,14.2,51.0,7200.0,5.9,0,0,0,0,0,0,0


利用KNN填補數值欄位空值


,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,1,1,0,0,121.0,36.0,1.2,137.6,4.20,15.4,44.0,7800.0,5.20,1,1,0,0,0,0,1
1,7.0,50.0,1.020,4.0,0.0,1,1,0,0,113.0,18.0,0.8,137.4,4.00,11.3,38.0,6000.0,4.96,0,0,0,0,0,0,1
2,62.0,80.0,1.010,2.0,3.0,1,1,0,0,423.0,53.0,1.8,133.8,4.20,9.6,31.0,7500.0,3.80,0,1,0,1,0,1,1
3,48.0,70.0,1.005,4.0,0.0,1,0,1,0,117.0,56.0,3.8,111.0,2.50,11.2,32.0,6700.0,3.90,1,0,0,1,1,1,1
4,51.0,80.0,1.010,2.0,0.0,1,1,0,0,106.0,26.0,1.4,138.4,3.98,11.6,35.0,7300.0,4.60,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,1,1,0,0,140.0,49.0,0.5,150.0,4.90,15.7,47.0,6700.0,4.90,0,0,0,0,0,0,0
396,42.0,70.0,1.025,0.0,0.0,1,1,0,0,75.0,31.0,1.2,141.0,3.50,16.5,54.0,7800.0,6.20,0,0,0,0,0,0,0
397,12.0,80.0,1.020,0.0,0.0,1,1,0,0,100.0,26.0,0.6,137.0,4.40,15.8,49.0,6600.0,5.40,0,0,0,0,0,0,0
398,17.0,60.0,1.025,0.0,0.0,1,1,0,0,114.0,50.0,1.0,135.0,4.90,14.2,51.0,7200.0,5.90,0,0,0,0,0,0,0


### 4(a.) 若要建構線性迴歸或羅吉斯迴歸分析，如何處理某些類別或名目尺度的欄位？

#### 在3(c.)的程式碼中已對類別或名目尺度的欄位做處理，將字串轉為1/0的編碼來表示

---
## Reference

- https://www.kaggle.com/code/mohannapd/99-accuracy-for-ckd-prediction/comments
- https://www.kaggle.com/code/yudhaislamisulistya/kidney-disease-knn-nbc-dt